In [1]:
#import libraries

import os
import random
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer

In [2]:
# load the dataset. it's in parquet format
df = pd.read_parquet("data/fhv_tripdata_2021-01.parquet")

In [3]:
print(f"Number of records in Jan 2021 FHV data {len(df)}")

Number of records in Jan 2021 FHV data 1154112


In [4]:
# compute the duration of each trip
df["duration"] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [5]:
# what's the average trip time?
trip_duration = np.mean(df.duration).round(2)
print(f"Average trip duration in January is {trip_duration} mins")

Average trip duration in January is 19.17 mins


In [6]:
def read_dataframe(filename):
    # load the dataset
    
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)
        
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
        df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
    else:
        df = pd.read_parquet(filename)
    
    # create a trip duration column
        
    df["duration"] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    # fill in the missing values with -1
    
    df["PUlocationID"].fillna(-1, inplace=True)
    df["DOlocationID"].fillna(-1, inplace=True)
    
    # PulocationID and DOlocation are categorical data
    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
  
    return df

In [7]:
# load the train and validatation data set. January is for training, February is for validation

train = read_dataframe("data/fhv_tripdata_2021-01.parquet")
val = read_dataframe("data/fhv_tripdata_2021-02.parquet")

In [8]:
# Explore the first five rows of our train data

train.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [9]:
# what's the percentage of the missing value in pick up location ID?

missing_values = np.around(train["PUlocationID"].value_counts(normalize=True), 3)
print(f"Fraction of missing values for pickup location ID: {missing_values[0] * 100} %")

Fraction of missing values for pickup location ID: 83.5 %


In [10]:
# one hot encoding of categorical variables using dictvectorizer

features = ["PUlocationID", "DOlocationID"]
dv = DictVectorizer()

train_dicts = train[features].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = val[features].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [11]:
#shape of the train and validation data after one hot encoding
print(f"The train dataset has {X_train.shape[0]} rows and {X_train.shape[1]} columns")
print(f"The validation dataset has {X_val.shape[0]} rows and {X_val.shape[1]} columns")

The train dataset has 1109826 rows and 525 columns
The validation dataset has 990113 rows and 525 columns


In [12]:
# create our features and target 

target = 'duration'
y_train = train[target].values
y_val = val[target].values

In [13]:
# train, fit and predict using LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

train_pred = lr.predict(X_train)

train_rmse = mean_squared_error(train_pred, y_train, squared=False)
print(f"Train rmse: {train_rmse}")

Train rmse: 10.528519425310185


In [14]:
# what is the performance of our model on validation data
y_pred = lr.predict(X_val)

val_rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"Val rmse: {val_rmse}")

Val rmse: 11.014285828610237


In [15]:
with open('models/lin_reg.bin', 'wb') as f:
    pickle.dump((dv, lr), f)